# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [1]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [2]:
from sklearn.model_selection import train_test_split

# Separate features (X) and target (Y)
X = adult_dt.drop(columns=['income'])
Y = adult_dt['income']

random_seed = 42

# Split data into training and testing sets (70-30% split)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=random_seed)

print("Data loaded and split successfully.")

Data loaded and split successfully.


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*

The *random_state* parameter in sklearn's `train_test_split` function is a seed that controls the randomness when splitting data into training and test sets.

It's useful for reproducibility - using the same random_state value will generate the same split every time, which is essential for:
- Debugging your ML pipeline
- Getting consistent results across experiments
- Allowing others to replicate your work exactly

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, OneHotEncoder

# List of numerical and categorical features
numerical_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_features = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

# Define the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        # Numerical features
        ('num_transforms', Pipeline([
            ('knn_imputer', KNNImputer(n_neighbors=7, weights="distance")),
            ('robust_scaler', RobustScaler())
        ]), numerical_features),
        
        # Categorical features
        ('cat_transforms', Pipeline([
            ('simple_imputer', SimpleImputer(strategy="most_frequent")),
            ('onehot', OneHotEncoder(handle_unknown="ignore", drop="if_binary"))
        ]), categorical_features)
    ]
)

print("Preprocessing pipeline created successfully.")

preprocessor

Preprocessing pipeline created successfully.


ColumnTransformer(transformers=[('num_transforms',
                                 Pipeline(steps=[('knn_imputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('robust_scaler',
                                                  RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat_transforms',
                                 Pipeline(steps=[('simple_imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'))]),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# Define the model pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),  # Use the Column Transformer created previously
    ('classifier', RandomForestClassifier(random_state=random_seed))  # Add the RandomForestClassifier as the final step
])

print("Model pipeline created successfully.")

model_pipeline

Model pipeline created successfully.


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('knn_imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('robust_scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('simple_imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [5]:
from sklearn.model_selection import cross_validate

# Define the scoring metrics
scoring = {
    'neg_log_loss': 'neg_log_loss',
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': 'balanced_accuracy'
}

# Perform cross-validation
cv_results = cross_validate(
    model_pipeline,    # The model pipeline defined earlier
    X_train,           # Training data features
    Y_train,           # Training data target
    scoring=scoring,   # Scoring metrics
    cv=5,              # Five-fold cross-validation
    return_train_score=True  # Report both training and validation results
)

# Display the cross-validation results
print("Cross-validation results:")
for metric in scoring.keys():
    print(f"{metric} - Training: {cv_results[f'train_{metric}'].mean():.4f} | Validation: {cv_results[f'test_{metric}'].mean():.4f}")


/home/msugar_dsi_2024_08_gmail_com/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/home/msugar_dsi_2024_08_gmail_com/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Cross-validation results:
neg_log_loss - Training: -0.0816 | Validation: -0.3680
roc_auc - Training: 1.0000 | Validation: 0.9033
accuracy - Training: 1.0000 | Validation: 0.8542
balanced_accuracy - Training: 1.0000 | Validation: 0.7763


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [6]:
# Convert the cross-validation results to a DataFrame
cv_results_df = pd.DataFrame(cv_results)

# Rename columns for clarity
cv_results_df = cv_results_df.rename(columns={
    'train_neg_log_loss': 'Train Neg Log Loss',
    'test_neg_log_loss': 'Test Neg Log Loss',
    'train_roc_auc': 'Train ROC AUC',
    'test_roc_auc': 'Test ROC AUC',
    'train_accuracy': 'Train Accuracy',
    'test_accuracy': 'Test Accuracy',
    'train_balanced_accuracy': 'Train Balanced Accuracy',
    'test_balanced_accuracy': 'Test Balanced Accuracy'
})

# Sort by the Test Neg Log Loss in ascending order (lower is better)
cv_results_df = cv_results_df.sort_values(by='Test Neg Log Loss', ascending=False)

# Display the DataFrame
cv_results_df.reset_index(drop=True, inplace=True)
cv_results_df


,fit_time,score_time,Test Neg Log Loss,Train Neg Log Loss,Test ROC AUC,Train ROC AUC,Test Accuracy,Train Accuracy,Test Balanced Accuracy,Train Balanced Accuracy
0,7.729880,0.178138,-0.356791,-0.082511,0.907250,1.0,0.859807,1.000000,0.782859,1.000000
1,7.665384,0.179806,-0.357675,-0.081189,0.904384,1.0,0.850625,0.999945,0.774484,0.999887
2,7.752791,0.178272,-0.369239,-0.081516,0.901079,1.0,0.850406,1.000000,0.771881,1.000000
3,7.691830,0.174648,-0.375988,-0.081469,0.901378,1.0,0.854103,0.999945,0.776017,0.999887
4,7.661165,0.176272,-0.380379,-0.081368,0.902299,1.0,0.856077,1.000000,0.776089,1.000000


Calculate the mean of each metric. 

In [7]:
# Calculate the mean for each metric
mean_metrics = cv_results_df.mean()

# Display the mean metrics
mean_metrics

fit_time                   7.700210
score_time                 0.177427
Test Neg Log Loss         -0.368014
Train Neg Log Loss        -0.081610
Test ROC AUC               0.903278
Train ROC AUC              1.000000
Test Accuracy              0.854204
Train Accuracy             0.999978
Test Balanced Accuracy     0.776266
Train Balanced Accuracy    0.999955
dtype: float64

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [8]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Fit the pipeline on the entire training data
model_pipeline.fit(X_train, Y_train)

# Predict probabilities and labels on the test data
Y_pred_proba = model_pipeline.predict_proba(X_test)
Y_pred = model_pipeline.predict(X_test)

# Calculate the performance metrics
test_metrics = {
    'Test Neg Log Loss': -log_loss(Y_test, Y_pred_proba),  # Neg log loss
    'Test ROC AUC': roc_auc_score(Y_test, Y_pred_proba[:, 1]),  # Only last column for ROC AUC
    'Test Accuracy': accuracy_score(Y_test, Y_pred),  # Accuracy
    'Test Balanced Accuracy': balanced_accuracy_score(Y_test, Y_pred)  # Balanced accuracy
}

# Display the results
test_metrics


{'Test Neg Log Loss': -0.3972650658939892,
 'Test ROC AUC': 0.8999828704291436,
 'Test Accuracy': 0.8551540587572934,
 'Test Balanced Accuracy': 0.7751631656838177}

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

Recoding the target variable `income` in the initial loading step to a binary format (0 for `<=50K` and 1 for `>50K`) is convenient for several reasons:

1. **Binary Encoding**: In many binary classification tasks, scikit-learn's evaluation metrics, such as accuracy, ROC AUC, and log loss, are designed to handle binary (0/1) targets seamlessly. Recoding `income` as 0 and 1 aligns it with the expected format, reducing the need for further transformations later on.

2. **Simplified Code and Consistency**: Encoding the target as a binary variable during data loading ensures that it is consistent across all subsequent steps. This avoids additional transformations within each model or evaluation function, keeping the code cleaner and easier to maintain.

3. **Efficient Performance of Classifiers**: Many algorithms in scikit-learn (e.g., `RandomForestClassifier`) work best with binary targets for binary classification tasks, where 0 represents the negative class and 1 represents the positive class. This approach minimizes ambiguity, streamlining both training and interpretation of model outputs.

By assigning a binary format early on, we align the dataset directly with scikit-learn’s default handling of binary classification, which simplifies the workflow and improves readability.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ x ] Created a branch with the correct naming convention.
- [ x ] Ensured that the repository is public.
- [ x ] Reviewed the PR description guidelines and adhered to them.
- [ x ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.